# Dining cryptographers

**Inspired by:** Chaum, D. (1988). _The dining cryptographers problem: Unconditional sender and recipient untraceability._ Journal of cryptology, 1, 65-75.

Three cryptographers are out for dinner. After dessert the waiter informs them that their meal was paid for anonymously. It was either paid for by one of the three cryptographers, or by the NSA. Yikes! To find out whether the bill was paid for by the NSA — without revealing which cryptographer paid, in case it wasn't the NSA — they carry out a protocol involving some coin tossing.

Each cryptographer tosses a coin and shows its outcome to their neighbor-to-the-left (hiding the coin behind their menus so the third cryptographer cannot see). Each cryptographer then announces the XOR of (1) their coin, (2) their neighbor's coin, and (3) whether or not they paid. Now, the XOR of all cryptographers' announcements reveals whether the NSA paid, without revealing which cryptographer (if any) paid.

In [1]:
from memo import memo
import jax
import jax.numpy as np
from enum import IntEnum

In [2]:
class Bit(IntEnum):
    NOT_PAID = 0
    PAID = 1

class Who(IntEnum):
    A_PAID = 0
    B_PAID = 1
    C_PAID = 2
    NSA_PAID = 3

We will model this from the perspective of cryptographer A (who didn't pay). We will show by computation that no matter how the coins come up and no matter what B and C announce, it is impossible to distinguish between B paying and C paying.

In [3]:
@memo
def model[a_: Bit, b_: Bit, bx: Bit, cx: Bit, w: Who]():
    a: knows(a_, w)
    a: thinks[
        world: chooses(w in Who, wpp=(w != 0)),
        b: chooses(b_ in Bit, wpp=1),
        c: chooses(c_ in Bit, wpp=1),

        b: knows(world.w, c.c_),
        c: knows(world.w, a_),

        b: chooses(bx in Bit, wpp=(b_ ^ c.c_ ^ (world.w == 1) == bx)),
        c: chooses(cx in Bit, wpp=(c_ ^   a_ ^ (world.w == 2) == cx)),
    ]
    a: observes [b.b_] is b_
    a: observes [b.bx] is bx
    a: observes [c.cx] is cx
    return a[Pr[world.w == w]]

Let's see what happens.

In [4]:
out = model()
%timeit -r 10 -n 100 out = model().block_until_ready()

import itertools
for a_, b_, bx, cx in itertools.product(Bit, Bit, Bit, Bit):
    print(f"A flips {a_}, B flips {b_}, B says {bx}, C says {cx} -> {out[a_, b_, bx, cx]}")

27.9 μs ± 2.33 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
A flips 0, B flips 0, B says 0, C says 0 -> [0. 0. 0. 1.]
A flips 0, B flips 0, B says 0, C says 1 -> [0.  0.5 0.5 0. ]
A flips 0, B flips 0, B says 1, C says 0 -> [0.  0.5 0.5 0. ]
A flips 0, B flips 0, B says 1, C says 1 -> [0. 0. 0. 1.]
A flips 0, B flips 1, B says 0, C says 0 -> [0.  0.5 0.5 0. ]
A flips 0, B flips 1, B says 0, C says 1 -> [0. 0. 0. 1.]
A flips 0, B flips 1, B says 1, C says 0 -> [0. 0. 0. 1.]
A flips 0, B flips 1, B says 1, C says 1 -> [0.  0.5 0.5 0. ]
A flips 1, B flips 0, B says 0, C says 0 -> [0.  0.5 0.5 0. ]
A flips 1, B flips 0, B says 0, C says 1 -> [0. 0. 0. 1.]
A flips 1, B flips 0, B says 1, C says 0 -> [0. 0. 0. 1.]
A flips 1, B flips 0, B says 1, C says 1 -> [0.  0.5 0.5 0. ]
A flips 1, B flips 1, B says 0, C says 0 -> [0. 0. 0. 1.]
A flips 1, B flips 1, B says 0, C says 1 -> [0.  0.5 0.5 0. ]
A flips 1, B flips 1, B says 1, C says 0 -> [0.  0.5 0.5 0. ]
A flips 1, B flips 1, B s

No matter what happens, A's posterior belief is always `[0 0 0 1]` (NSA paid) or `[0 .5 .5 0]` (B or C paid, but unsure which).